In [15]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

NUM_GAMES = 380
NUM_WEEKS = 38

prev_perf_name = 'data/prev_perf.csv'
csv_name = 'data/E0.csv'

df = pd.read_csv(csv_name)
df_prev = pd.read_csv(prev_perf_name, header=None)
df_prev.head()

#df.head()

,0,1,2,3
0,Arsenal,75,83,38
1,Aston Villa,38,31,57
2,Bournemouth,30,42,73
3,Chelsea,87,73,32
4,Crystal Palace,48,47,51


In [11]:
uniq_team_lst = list(df.HomeTeam.unique())
uniq_team_lst.sort()

home_team_ids = df.HomeTeam.apply(
    lambda x: uniq_team_lst.index(x)
) 


away_team_ids = df.AwayTeam.apply(
    lambda x: uniq_team_lst.index(x) 
)

score_diff = df.FTHG - df.FTAG


In [20]:
home_team_wk = [0] * NUM_GAMES
away_team_wk = [0] * NUM_GAMES

for i in range(NUM_GAMES):
    home_team_wk[i] = sum(home_team_ids[:i+1] == home_team_ids[i]) + sum(
    away_team_ids[:i+1] == home_team_ids[i])
    
    away_team_wk[i] = sum(home_team_ids[:i+1] == away_team_ids[i]) + sum(
    away_team_ids[:i+1] == away_team_ids[i])


    1   2   3
0  75  83  38
1  38  31  57
2  30  42  73


In [22]:
prev_perf = preprocessing.minmax_scale(df_prev.iloc[:, 1], feature_range=(-1,1))
print(prev_perf)

[ 0.57894737 -0.71929825 -1.          1.         -0.36842105 -0.40350877
 -0.61403509  0.12280702  0.71929825  0.40350877 -0.68421053 -1.
  0.05263158 -0.15789474 -0.71929825 -0.0877193   0.19298246 -1.
 -0.50877193 -0.40350877]
